In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import json
from datetime import datetime, timedelta

In [2]:
date_list = []
# Create an empty list to store the dictionaries
top_chart_list = []

In [3]:
start_date = datetime.strptime('2024-06-08', '%Y-%m-%d')
end_date = datetime.strptime('2023-09-16', '%Y-%m-%d')

current_date = start_date
while current_date >= end_date:
    print(current_date.strftime('%Y-%m-%d'))
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date -= timedelta(days=7)

2024-06-08
2024-06-01
2024-05-25
2024-05-18
2024-05-11
2024-05-04
2024-04-27
2024-04-20
2024-04-13
2024-04-06
2024-03-30
2024-03-23
2024-03-16
2024-03-09
2024-03-02
2024-02-24
2024-02-17
2024-02-10
2024-02-03
2024-01-27
2024-01-20
2024-01-13
2024-01-06
2023-12-30
2023-12-23
2023-12-16
2023-12-09
2023-12-02
2023-11-25
2023-11-18
2023-11-11
2023-11-04
2023-10-28
2023-10-21
2023-10-14
2023-10-07
2023-09-30
2023-09-23
2023-09-16


In [4]:
x = 0
browser = Browser('chrome')
for date in date_list:
    url = 'https://www.billboard.com/charts/tiktok-billboard-top-50/' + date + "/"
    browser.visit(url)
    # Create a Beautiful Soup object
    html = browser.html
    soup_info = soup(html, 'html.parser')
    # Extract all the text elements
    Top_chart_elements = soup_info.find_all("div", class_="o-chart-results-list-row-container")
    # Loop through the text elements
    for article in Top_chart_elements:
    
        date = date
        title = article.find("h3", class_="c-title").get_text()
        try:
            artist = article.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet").get_text()
        except:
            artist = artist = article.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only").get_text()

    
        track_name = title.strip()
        artist_name = artist.strip()
    
        # Store in a dictionary
        article_dict = {
            "title": track_name,
            "artist": artist_name,
            "date": date
            }
    
        # Append the dictionary to the list
        top_chart_list.append(article_dict)
    # x+=1
    # if x ==3 : 
    #     browser.quit()
    #     break

In [5]:
browser.quit()

In [6]:
top_chart_df = pd.DataFrame(top_chart_list)
top_chart_df

,title,artist,date
0,Million Dollar Baby,Tommy Richman,2024-06-08
1,Tell Ur Girlfriend,Lay Bankz,2024-06-08
2,U My Everything,Sexyy Red & Drake,2024-06-08
3,Birds Of A Feather,Billie Eilish,2024-06-08
4,Not Like Us,Kendrick Lamar,2024-06-08
...,...,...,...
1945,XXL,LANY,2023-09-16
1946,Falling Behind,Laufey,2023-09-16
1947,Power Trip,J. Cole Featuring Miguel,2023-09-16
1948,Beautiful,Bazzi Featuring Camila Cabello,2023-09-16


In [7]:
top_chart_df_copy2 = top_chart_df.copy()

In [8]:
import requests

CLIENT_ID = "your_id_here"
CLIENT_SECRET = "your_secret_here"

AUTH_URL = "https://accounts.spotify.com/api/token"
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': "68eea273ba0b4954a6237107625186fe",
    'client_secret': "18597e9724e945c8a64f86a2093aaea3",
})

#Convert response to JSON
auth_response_data = auth_response.json()

#Save the access token
access_token = auth_response_data['access_token']

#Need to pass access token into header to send properly formed GET request to API server
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [9]:
artist = "Tommy Richman"
trackName = "Million Dollar Baby"

In [10]:
BASE_URL = 'https://api.spotify.com/v1/'
r = requests.get(BASE_URL + f'search?q={artist} {title}&type=artist,track&offset=0&limit=1', headers=headers)
r = r.json()


In [11]:
r

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Tommy+Richman+%0A%0A%09%0A%09%0A%09%09%0A%09%09%09%09%09Mad+At+Me%09%09%0A%09%0A&type=artist&offset=0&limit=1',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1WaFQSHVGZQJTbf0BdxdNo'},
    'followers': {'href': None, 'total': 430280},
    'genres': ['chill abstract hip hop'],
    'href': 'https://api.spotify.com/v1/artists/1WaFQSHVGZQJTbf0BdxdNo',
    'id': '1WaFQSHVGZQJTbf0BdxdNo',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5ebbb330168f192dbdc28fcbd37',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab67616100005174bb330168f192dbdc28fcbd37',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f178bb330168f192dbdc28fcbd37',
      'width': 160}],
    'name': 'Tommy Richman',
    'popularity': 81,
    'type': 'artist',
    'uri': 'spotify:artist:1WaFQSHVGZQJTbf0BdxdNo'}],
  'limit': 1,
  '

In [12]:
track_info = r['tracks']['items'][0]['id']

In [13]:
track_info

'5AJ9hqTS2wcFQCELCFRO7A'

In [14]:
BASE_URL = 'https://api.spotify.com/v1/'

URI_list = []

for i, row in top_chart_df.iterrows():
    track = row[0]
    artist = row[1]
    r = requests.get(BASE_URL + f'search?q={artist}+{track}&type=artist,track&offset=0&limit=1', headers=headers)
    info = r.json()
    URI = info['tracks']['items'][0]['id']
    URI_list.append(URI)

URI_list

C:\Users\Jack Runge\AppData\Local\Temp\ipykernel_8520\1445716027.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  track = row[0]
C:\Users\Jack Runge\AppData\Local\Temp\ipykernel_8520\1445716027.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  artist = row[1]


['5AJ9hqTS2wcFQCELCFRO7A',
 '3lMzT16MjAKKXF7pSZn13B',
 '2MjXWroB9wlTG2kqv3avfS',
 '6dOtVTDdiauQNBQEDOtlAB',
 '6AI3ezQ4o3HUoP6Dhudph3',
 '4PhTvtwhgfkn9jlgPEBbue',
 '28drn6tQo95MRvO0jQEo5C',
 '7iabz12vAuVQYyekFIWJxD',
 '1C84d9abZVKWHT2YYpoean',
 '6XjDF6nds4DE2BBbagZol6',
 '6NjWCIYu1W8xa3HIvcIhd4',
 '1aHy1ipWQRNauI0ee9uYfi',
 '6Uwi2Qk3H7fM4b4W4ExrAp',
 '2FQrifJ1N335Ljm3TjTVVf',
 '3KjRNFs97uhUD2ssfU1coJ',
 '4ZJ4vzLQekI0WntDbanNC7',
 '29TPjc8wxfz4XMn21O7VsZ',
 '629DixmZGHc7ILtEntuiWE',
 '2qSkIjg1o9h3YT9RAgYN75',
 '5rcnAHBclBs0OGV4rATEnR',
 '2GxrNKugF82CnoRFbQfzPf',
 '4HnPZXkx15Y5hfBWh9vtQL',
 '6O3WfmAQIgnLBGVzZJVS40',
 '2tudvzsrR56uom6smgOcSf',
 '22yRHdYBLZMi7xnvBioqkP',
 '1i8dJGpKO0xQiKGCVslJqB',
 '0mIdZEa3kU6btHfzbEibz4',
 '3hFnQwSt5zAWaYpLuMSWB7',
 '7rIhp6EWLNtM8qFIQruJPT',
 '48lxT5qJF0yYyf2z4wB4xW',
 '3QaPy1KgI7nu9FJEQUgn6h',
 '2prqm9sPLj10B4Wg0wE5x9',
 '0e1KTuawmiFLiK0Lh3nNtM',
 '2V852FRIFO7mFgfiyDMum1',
 '37adYGaYaAWTGhBaOzX4Fh',
 '6sghEq6gM4Ugadwa12H7Sa',
 '5UNzhuQS8ak02kW0EXsI24',
 

In [24]:
index = 0
temp_df = pd.DataFrame()
for URI in URI_list:
    Track_URL = f'https://api.spotify.com/v1/audio-features/{URI}'
    track_info = requests.get(Track_URL, headers=headers)
    track_details = track_info.json()
    print(track_details)
    df = pd.DataFrame(track_details, index = [index])
    df = df.drop(columns=['type', 'id', 'track_href', 'analysis_url', 'duration_ms', 'time_signature' ])
    index+=1
    temp_df = pd.concat([temp_df, df])

{'error': {'status': 429}}


KeyError: "['type', 'id', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'] not found in axis"

In [22]:
len(URI_list)

1950

In [16]:
temp_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri
0,0.852,0.697,1,-5.520,0,0.0439,0.0973,0.000370,0.0678,0.919,138.029,spotify:track:5AJ9hqTS2wcFQCELCFRO7A
1,0.866,0.741,7,-4.660,1,0.2450,0.0933,0.000000,0.0297,0.614,135.070,spotify:track:3lMzT16MjAKKXF7pSZn13B
2,0.811,0.640,5,-5.630,0,0.2740,0.0957,0.000000,0.1270,0.527,145.031,spotify:track:2MjXWroB9wlTG2kqv3avfS
3,0.747,0.507,2,-10.171,1,0.0358,0.2000,0.060800,0.1170,0.438,104.978,spotify:track:6dOtVTDdiauQNBQEDOtlAB
4,0.898,0.472,1,-7.001,1,0.0776,0.0107,0.000000,0.1410,0.214,101.061,spotify:track:6AI3ezQ4o3HUoP6Dhudph3
...,...,...,...,...,...,...,...,...,...,...,...,...
1251,0.336,0.627,7,-7.463,1,0.0384,0.1640,0.000000,0.0708,0.350,150.273,spotify:track:0bYg9bo50gSsH3LtXe2SQn
1252,0.846,0.738,3,-4.537,0,0.0803,0.2340,0.000000,0.0987,0.949,102.978,spotify:track:1S2WWT5jW9J3MauQ7r7qN4
1253,0.504,0.308,9,-14.958,1,0.0321,0.8680,0.135000,0.1580,0.121,113.950,spotify:track:3vkCueOmm7xQDoJ17W1Pm3
1254,0.943,0.558,2,-4.911,1,0.0568,0.0026,0.000002,0.0937,0.606,104.983,spotify:track:4xhsWYTOGcal8zt0J161CU


In [ ]:
top_chart_df_copy2 = pd.concat([top_chart_df_copy2, temp_df], axis=1)
top_chart_df_copy2